# Random Forest Classification

## Importing the libraries

In [1]:
import random
import numpy as np
import pandas as pd
import seaborn as sn
from numpy import std
from numpy import mean
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score
%matplotlib inline

In [2]:
%cd Z:\PPMI_Data\Excels\Only_Male\GeneCpG\Fatigue
df = pd.read_csv('FatigueCateg_Methylome_Male.csv')
print(df.shape)
df.head(3)

Z:\PPMI_Data\Excels\Only_Male\GeneCpG\Fatigue
(307, 31218)


,Unnamed: 0,Sentrix,PATNO,HYS,Gender,APPRDX,MDSP_Fatigue,cg17960051,cg06562372,cg04099454,...,cg15500865,cg10329418,cg18241308,cg04350520,cg04685170,cg20985067,cg08461772,cg25879118,cg07510163,ch.13.693633F
0,0,200991620021_R01C01,3001,2,1,1,1,0.019716,0.024434,0.895146,...,0.105174,0.262518,0.784256,0.886425,0.080717,0.814912,0.331552,0.916387,0.841837,0.036872
1,1,200973410167_R04C01,3007,2,1,1,1,0.020417,0.016055,0.882493,...,0.066103,0.210723,0.905081,0.882225,0.082907,0.906400,0.497660,0.915460,0.822889,0.039866
2,2,200973410167_R05C01,3010,2,1,1,1,0.020654,0.017736,0.927835,...,0.330337,0.310647,0.852778,0.876989,0.183659,0.917887,0.470535,0.894010,0.823575,0.036319


In [3]:
df1 = df.drop(['Unnamed: 0','PATNO', 'Sentrix', 'HYS', 'Gender', 'MDSP_Fatigue'], axis=1)
df1.head(2)

,APPRDX,cg17960051,cg06562372,cg04099454,cg18542829,cg04599912,cg15759056,cg16703576,cg00031165,cg26835636,...,cg15500865,cg10329418,cg18241308,cg04350520,cg04685170,cg20985067,cg08461772,cg25879118,cg07510163,ch.13.693633F
0,1,0.019716,0.024434,0.895146,0.024184,0.014927,0.012150,0.026345,0.078477,0.016245,...,0.105174,0.262518,0.784256,0.886425,0.080717,0.814912,0.331552,0.916387,0.841837,0.036872
1,1,0.020417,0.016055,0.882493,0.045616,0.021537,0.013626,0.032826,0.033160,0.018156,...,0.066103,0.210723,0.905081,0.882225,0.082907,0.906400,0.497660,0.915460,0.822889,0.039866


In [4]:
df1 = df1.transpose()
df1.head(2)

,0,1,2,3,4,5,6,7,8,9,...,297,298,299,300,301,302,303,304,305,306
APPRDX,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,2.000000,2.00000,2.000000,2.00000,2.000000,2.00000,2.000000,2.000000,2.000000,2.000000
cg17960051,0.019716,0.020417,0.020654,0.015615,0.019431,0.018499,0.017311,0.015358,0.015278,0.022692,...,0.017515,0.01611,0.016656,0.01624,0.011848,0.01539,0.019081,0.015741,0.025337,0.018544


In [5]:
df1.rename(columns=df1.iloc[0], inplace = True)


In [6]:
df1.index.name ='CpG'
df1.reset_index(inplace=True)
df1.head(2)

,CpG,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
0,APPRDX,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,2.000000,2.00000,2.000000,2.00000,2.000000,2.00000,2.000000,2.000000,2.000000,2.000000
1,cg17960051,0.019716,0.020417,0.020654,0.015615,0.019431,0.018499,0.017311,0.015358,0.015278,...,0.017515,0.01611,0.016656,0.01624,0.011848,0.01539,0.019081,0.015741,0.025337,0.018544


In [7]:
df1 = df1.drop ([0], axis = 0)
df1.head(2)

,CpG,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,cg17960051,0.019716,0.020417,0.020654,0.015615,0.019431,0.018499,0.017311,0.015358,0.015278,...,0.017515,0.016110,0.016656,0.016240,0.011848,0.015390,0.019081,0.015741,0.025337,0.018544
2,cg06562372,0.024434,0.016055,0.017736,0.079565,0.013873,0.018993,0.034363,0.029264,0.045489,...,0.082414,0.043832,0.026010,0.015081,0.071585,0.039357,0.066754,0.014234,0.008521,0.022749


In [8]:
df2 = pd.read_csv ("Fatigue_NMI_Top50_Male.csv")
df2.head(2)

,CpG
0,cg09347135
1,cg25772658


In [9]:
df3 = pd.merge (df1, df2, on= "CpG", how= "inner")
df3.head(2)

,CpG,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
0,cg03787486,0.035693,0.034595,0.035085,0.026821,0.023798,0.027742,0.030956,0.033423,0.031603,...,0.029540,0.034092,0.030423,0.037258,0.022528,0.020184,0.035927,0.037106,0.042316,0.030269
1,cg04721711,0.831384,0.875973,0.876690,0.888754,0.878808,0.909823,0.884314,0.891160,0.880420,...,0.894598,0.880789,0.899124,0.883628,0.883107,0.893757,0.891451,0.884865,0.853245,0.869702


In [10]:
df3 = df3.transpose()
df3.head(2)

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
CpG,cg03787486,cg04721711,cg05826330,cg02295000,cg09537714,cg15272034,cg24810998,cg16448286,cg12350196,cg17240004,...,cg16338844,cg06470379,cg15228932,cg25330422,cg17136384,cg10538320,cg16195239,cg25772658,cg09826790,cg19095988
1.0,0.035693,0.831384,0.015721,0.826318,0.72633,0.850724,0.73443,0.906014,0.407146,0.922325,...,0.753305,0.77522,0.922735,0.713645,0.901962,0.032919,0.946079,0.911458,0.86927,0.75717


In [11]:
df3.rename(columns=df3.iloc[0], inplace = True)


In [12]:
df3.index.name ='APPRDX'
df3.reset_index(inplace=True)
df3.head(2)

,APPRDX,cg03787486,cg04721711,cg05826330,cg02295000,cg09537714,cg15272034,cg24810998,cg16448286,cg12350196,...,cg16338844,cg06470379,cg15228932,cg25330422,cg17136384,cg10538320,cg16195239,cg25772658,cg09826790,cg19095988
0,CpG,cg03787486,cg04721711,cg05826330,cg02295000,cg09537714,cg15272034,cg24810998,cg16448286,cg12350196,...,cg16338844,cg06470379,cg15228932,cg25330422,cg17136384,cg10538320,cg16195239,cg25772658,cg09826790,cg19095988
1,1.0,0.035693,0.831384,0.015721,0.826318,0.72633,0.850724,0.73443,0.906014,0.407146,...,0.753305,0.77522,0.922735,0.713645,0.901962,0.032919,0.946079,0.911458,0.86927,0.75717


In [13]:
df3 = df3.drop ([0], axis = 0)
df3.head(2)

,APPRDX,cg03787486,cg04721711,cg05826330,cg02295000,cg09537714,cg15272034,cg24810998,cg16448286,cg12350196,...,cg16338844,cg06470379,cg15228932,cg25330422,cg17136384,cg10538320,cg16195239,cg25772658,cg09826790,cg19095988
1,1.0,0.035693,0.831384,0.015721,0.826318,0.72633,0.850724,0.73443,0.906014,0.407146,...,0.753305,0.77522,0.922735,0.713645,0.901962,0.032919,0.946079,0.911458,0.86927,0.75717
2,1.0,0.034595,0.875973,0.018893,0.805755,0.742926,0.837866,0.907744,0.890996,0.345335,...,0.769894,0.840957,0.91682,0.655201,0.872775,0.025642,0.905118,0.903341,0.914964,0.849966


In [14]:
df3.reset_index(inplace = True, drop = True)
df3.head(3)

,APPRDX,cg03787486,cg04721711,cg05826330,cg02295000,cg09537714,cg15272034,cg24810998,cg16448286,cg12350196,...,cg16338844,cg06470379,cg15228932,cg25330422,cg17136384,cg10538320,cg16195239,cg25772658,cg09826790,cg19095988
0,1.0,0.035693,0.831384,0.015721,0.826318,0.72633,0.850724,0.73443,0.906014,0.407146,...,0.753305,0.77522,0.922735,0.713645,0.901962,0.032919,0.946079,0.911458,0.86927,0.75717
1,1.0,0.034595,0.875973,0.018893,0.805755,0.742926,0.837866,0.907744,0.890996,0.345335,...,0.769894,0.840957,0.91682,0.655201,0.872775,0.025642,0.905118,0.903341,0.914964,0.849966
2,1.0,0.035085,0.87669,0.02127,0.79669,0.72021,0.841361,0.888154,0.88818,0.28549,...,0.768303,0.836399,0.914683,0.694116,0.855246,0.021741,0.898667,0.895997,0.894834,0.842801


In [15]:
df3.shape

(307, 51)

In [16]:
df3 ['APPRDX'] = df3 ['APPRDX'].apply(int)

## RanFor 1000 times, CpG only 

In [17]:
df3.head(2)

,APPRDX,cg03787486,cg04721711,cg05826330,cg02295000,cg09537714,cg15272034,cg24810998,cg16448286,cg12350196,...,cg16338844,cg06470379,cg15228932,cg25330422,cg17136384,cg10538320,cg16195239,cg25772658,cg09826790,cg19095988
0,1,0.035693,0.831384,0.015721,0.826318,0.72633,0.850724,0.73443,0.906014,0.407146,...,0.753305,0.77522,0.922735,0.713645,0.901962,0.032919,0.946079,0.911458,0.86927,0.75717
1,1,0.034595,0.875973,0.018893,0.805755,0.742926,0.837866,0.907744,0.890996,0.345335,...,0.769894,0.840957,0.91682,0.655201,0.872775,0.025642,0.905118,0.903341,0.914964,0.849966


In [18]:
df3.shape

(307, 51)

In [ ]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,1001) :
        print(i)
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(90))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe1 = dframe1.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
dframe1.head(2)

In [ ]:
dframe1.to_csv ("Fatigue_NMICpG_Male_1000RF.csv", index=False)